## **1. Télécharger la base d'images**
https://cluster.ig.umons.ac.be/workshop_ia/image.orig.zip 
 

## **2. Télécharger les descriteurs (exemples de descripteurs obtenues avec VGG16)**
https://drive.google.com/drive/folders/1--t1vVUrXnU3DByxCyG9Fs2rxwXRWlGr?usp=sharing  

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications import vgg16
from keras.applications import vgg19
from keras.applications import resnet50
from keras.applications import inception_v3
from keras.applications import mobilenet
from keras.applications import xception
#from scipy.misc import imread, imresize
from matplotlib.pyplot import imread
import matplotlib.pyplot as plt
import numpy as np
import operator
import math
import os
import tensorflow as tf

In [ ]:
def euclidianDistance(l1,l2):
    distance = 0
    length = min(len(l1),len(l2))
    for i in range(length):
        distance += pow((l1[i] - l2[i]), 2)
    return math.sqrt(distance)

In [ ]:
def getkVoisins(lfeatures, test, k) : 
    ldistances = []
    for i in range(len(lfeatures)):
        dist = euclidianDistance(test[1], lfeatures[i][1])
        ldistances.append((lfeatures[i][0], lfeatures[i][1], dist))
    ldistances.sort(key=operator.itemgetter(2))
    lvoisins = []
    for i in range(k):
        lvoisins.append(ldistances[i])
    return lvoisins

In [ ]:
from keras.models import Model

In [ ]:
model0 = vgg16.VGG16(weights='imagenet', include_top=True,pooling='avg')
model1 = Model(inputs=model0.input, outputs=model0.layers[-2].output)

553467904/553467096 [==============================] - 4s 0us/step


In [ ]:
files='./image.orig'
folder_model = './VGG16'
features1=[]
pas=0
for j in os.listdir(folder_model): #Le paramètre "files" donne le nom du dossier dans lequel se trouvent les images
  data=os.path.join(folder_model,j)
  if not data.endswith(".txt"):
    continue         
  feature = np.loadtxt(data)
  features1.append((os.path.join(files,os.path.basename(data).split('.')[0]+'.jpg'),feature))
  #print(pas)
  pas += 1

In [ ]:
import warnings
warnings.filterwarnings('ignore')
sortie=20
voisins = getkVoisins(features1, features1[123], sortie)

nom_image_plus_proches = []
nom_image_plus_proches_sans = [] 

for k in range(sortie):
    nom_image_plus_proches.append(voisins[k][0])
plt.figure(figsize=(5, 5))
plt.imshow(imread(features1[123][0]), cmap='gray', interpolation='none')
plt.title("Image requête")
nom_image_requete=os.path.splitext(os.path.basename(features1[123][0]))[0]
print(nom_image_requete)
      
plt.figure(figsize=(25, 25))
plt.subplots_adjust(hspace=0.2, wspace=0.2)

for j in range(sortie): 
    plt.subplot(sortie/4,sortie/5,j+1)
    plt.imshow(imread(nom_image_plus_proches[j]), cmap='gray', interpolation='none')
    nom_image_plus_proches_sans.append(os.path.splitext(os.path.basename(nom_image_plus_proches[j]))[0])
    title = "Image proche n°"+str(j)
    plt.title(title)

In [ ]:
text_file = open("VGG_RP.txt", "w")
rappel_precision=[]
rp = []
position1=int(nom_image_requete)//100
for j in range(sortie):
  position2=int(nom_image_plus_proches_sans[j])//100
  if position1==position2:
    rappel_precision.append("pertinant")
  else:
    rappel_precision.append("non pertinant")

for i in range(sortie):
  j=i
  val=0
  while j>=0:
    if rappel_precision[j]=="pertinant":
      val+=1
    j-=1
  rp.append(str((val/(i+1))*100)+" "+str((val/sortie)*100))
  
with open("VGG_RP.txt", 'w') as s:
  for a in rp:
    s.write(str(a) + '\n')  

In [ ]:
import csv
x = []
y = []
fichier = "VGG_RP.txt"
with open(fichier) as csvfile:
    plots = csv.reader(csvfile, delimiter=' ')
    for row in plots:
        x.append(float(row[0]))
        y.append(float(row[1]))
        fig = plt.figure()
line, =plt.plot(y,x,'C1', label="VGG16" )
plt.xlabel('Rappel')
plt.ylabel('Précison')
plt.title("R/P")
plt.legend()